In [18]:
import os, subprocess, shutil, re
import pandas as pd

In [5]:
input_dir = '../GraphSeg/data/input'
output_dir = '../GraphSeg/data/output'
seg_en = '../GraphSeg/binary/graphseg_en.jar'
seg_nl = '../GraphSeg/binary/graphseg_nl.jar'

#### ENWiki tests

In [14]:
# Copy files to input_orig directory
for root, _, files in os.walk('../ENWiki/data'):
    for file in files[:5]:
        if not os.path.exists(os.path.join(input_dir + '_orig', file)):
            shutil.copy(os.path.join(root, file), input_dir + '_orig')
    break

In [ ]:
# Clean up docs, place them in input directory
for root, _, files in os.walk(input_dir + '_orig'):
    for file in files:
        with open(os.path.join(root, file), 'r') as f:
            doc = f.read()
        with open(os.path.join(input_dir, file), 'w') as f2:
            f2.write(re.sub(r'^=+.*\n+', '', doc, flags=re.MULTILINE))

In [ ]:
# Call the jar (and 🙏)
with subprocess.Popen(['java', '-jar', seg_en, input_dir, output_dir, '0.25', '2'], stdout=subprocess.PIPE, universal_newlines=True) as popen:
    for line in popen.stdout:
        print(line)
if popen.returncode != 0:
    raise subprocess.CalledProcessError(popen.returncode, popen.args)

#### NLWiki tests
Dutch alternatives:
* https://github.com/dcferreira/multilingual-joint-embeddings for word embeddings (it is based on what's used by the English version)
* http://crr.ugent.be/programs-data/subtitle-frequencies/subtlex-nl/downloading for word frequencies
* `nltk.corpus.stopwords.words('dutch')` for stopwords

##### Creating replacements for the English resources

In [ ]:
# Extract correct word embeddings


In [30]:
# Transform word frequency csv into correct .txt format
df = pd.read_csv('../GraphSeg/backups/SUBTLEX-NL.cd-above2.txt', sep='\t', header=0)
out_str = df.iloc[:, :2].to_string(header=False, index=False, justify='left')
with open('../GraphSeg/source/res/freqs.txt', 'w') as out:
    out.write(re.sub(r'[ \t]+', ' ', out_str))

In [45]:
# Get stopwords from nltk and put them into a .txt file
from nltk.corpus import stopwords
nl_sw = stopwords.words('dutch')
with open('../GraphSeg/source/res/stopwords.txt', 'w') as out:
    out.write('\n'.join(nl_sw))

##### Testing

In [ ]:
for root, _, files in os.walk('../NLWiki/data'):
    for file in files[:5]:
        if not os.path.exists(os.path.join(input_dir + '_orig', file)):
            shutil.copy(os.path.join(root, file), input_dir + '_orig')
    break